In [15]:
import pandas as pd
import altair as alt

In [16]:
df = pd.read_csv('data/SP500_merged.csv')
df.head()

,Date,Close,Volume,Symbol,GICS Sector,GICS Sub-Industry
0,2022-01-03 00:00:00-05:00,177.740005,1930700,MMM,Industrials,Industrial Conglomerates
1,2022-01-04 00:00:00-05:00,180.229996,2522200,MMM,Industrials,Industrial Conglomerates
2,2022-01-05 00:00:00-05:00,179.490005,2952400,MMM,Industrials,Industrial Conglomerates
3,2022-01-06 00:00:00-05:00,178.000000,2505400,MMM,Industrials,Industrial Conglomerates
4,2022-01-07 00:00:00-05:00,179.949997,2800200,MMM,Industrials,Industrial Conglomerates


In [17]:
# Obtain the sector name as a list, total 11 sectors
sector_name = df['GICS Sector'].unique().tolist()
sector_name

['Industrials',
 'Health Care',
 'Information Technology',
 'Communication Services',
 'Consumer Staples',
 'Consumer Discretionary',
 'Utilities',
 'Financials',
 'Materials',
 'Real Estate',
 'Energy']

In [18]:
# Obtain the symbol of each sector as a list
sector_symbol = df['Symbol'].unique().tolist()
#sector_symbol

In [19]:
# Define a function to calculate the growth rate of each company (sector_symbol) in each sector (sector_name)
def growth_rate_company(duration, df):
    # duration: number of days to calculate the growth rate
    # df: the dataframe to be used
    df['Date'] = pd.to_datetime(df['Date'], utc=True)
    end_date = df['Date'].max()
    start_date = end_date - pd.DateOffset(days = duration)
    dff = df[df['Date'].between(start_date, end_date)]

    # Calculate the growth rate of each company in each sector
    company_growth_rate = dff.groupby(['GICS Sector', 'Symbol']).apply(lambda x: (x['Close'].iloc[-1] - x['Close'].iloc[0])/x['Close'].iloc[0]).reset_index(name='Growth Rate')

    return company_growth_rate

In [20]:
growth_rate_company(90, df)

,GICS Sector,Symbol,Growth Rate
0,Communication Services,ATVI,0.033469
1,Communication Services,CHTR,-0.066268
2,Communication Services,CMCSA,0.016785
3,Communication Services,DIS,0.021764
4,Communication Services,DISH,-0.300312
...,...,...,...
496,Utilities,PPL,-0.067920
497,Utilities,SO,-0.052853
498,Utilities,SRE,-0.087250
499,Utilities,WEC,-0.091840


In [21]:
# Define a function to subset the original df to only include the top 5 companies in each sector
def top_5_company(duration, df):

    company_growth_rate = growth_rate_company(duration, df)

    # Filter the top 5 companies in each sector
    top_5 = company_growth_rate.groupby('GICS Sector').apply(lambda x: x.nlargest(5, 'Growth Rate'))
    top_5 = top_5.reset_index(drop=True)

    # Obtain the symbol of top 5 companies in each sector
    top_5_symbol = top_5['Symbol'].unique().tolist()

    # Subset the df to only include the top 5 companies in each sector
    df_top_5 = df[df['Symbol'].isin(top_5_symbol)]

    return df_top_5

In [22]:
df_top_5 = top_5_company(90, df)
df_top_5

,Date,Close,Volume,Symbol,GICS Sector,GICS Sub-Industry
5220,2022-01-03 05:00:00+00:00,648.049988,428200,ALGN,Health Care,Health Care Supplies
5221,2022-01-04 05:00:00+00:00,623.500000,552000,ALGN,Health Care,Health Care Supplies
5222,2022-01-05 05:00:00+00:00,578.840027,980000,ALGN,Health Care,Health Care Supplies
5223,2022-01-06 05:00:00+00:00,567.619995,1149400,ALGN,Health Care,Health Care Supplies
5224,2022-01-07 05:00:00+00:00,546.989990,1190500,ALGN,Health Care,Health Care Supplies
...,...,...,...,...,...,...
143002,2023-02-22 05:00:00+00:00,107.669998,3375500,WYNN,Consumer Discretionary,Casinos & Gaming
143003,2023-02-23 05:00:00+00:00,105.839996,1820700,WYNN,Consumer Discretionary,Casinos & Gaming
143004,2023-02-24 05:00:00+00:00,105.529999,1942900,WYNN,Consumer Discretionary,Casinos & Gaming
143005,2023-02-27 05:00:00+00:00,105.139999,2259000,WYNN,Consumer Discretionary,Casinos & Gaming


In [23]:
# check number of rows for each sector
df_top_5['GICS Sector'].value_counts()


Information Technology    1450
Financials                1450
Energy                    1450
Consumer Discretionary    1450
Real Estate               1450
Materials                 1450
Consumer Staples          1450
Utilities                 1450
Communication Services    1450
Industrials               1450
Health Care               1210
Name: GICS Sector, dtype: int64

In [40]:
# Subset the df_top_5 based on the GICS Sector

# for example, if we want to see the top 5 companies in Communication Services sector
df_top_5_communication = df_top_5[df_top_5['GICS Sector']=='Communication Services']
df_top_5_communication

,Date,Close,Volume,Symbol,GICS Sector,GICS Sub-Industry
59439,2022-01-03 05:00:00+00:00,37.639999,2917300,FOXA,Communication Services,Movies & Entertainment
59440,2022-01-04 05:00:00+00:00,37.970001,4720500,FOXA,Communication Services,Movies & Entertainment
59441,2022-01-05 05:00:00+00:00,37.610001,2024200,FOXA,Communication Services,Movies & Entertainment
59442,2022-01-06 05:00:00+00:00,38.430000,2716700,FOXA,Communication Services,Movies & Entertainment
59443,2022-01-07 05:00:00+00:00,39.020000,4427700,FOXA,Communication Services,Movies & Entertainment
...,...,...,...,...,...,...
138942,2023-02-22 05:00:00+00:00,15.420000,20901300,WBD,Communication Services,Broadcasting
138943,2023-02-23 05:00:00+00:00,15.730000,27591400,WBD,Communication Services,Broadcasting
138944,2023-02-24 05:00:00+00:00,15.550000,38605700,WBD,Communication Services,Broadcasting
138945,2023-02-27 05:00:00+00:00,15.940000,20744900,WBD,Communication Services,Broadcasting


In [25]:
# Define the sectors and their symbols
sector_namedf_top_5 = df_top_5['GICS Sector'].unique().tolist()

symbols_by_sectordf_top_5 = {sector: df_top_5[df_top_5['GICS Sector'] == sector]['Symbol'].unique() for sector in sector_name}

symbols_by_sectordf_top_5

{'Industrials': array(['GE', 'PH', 'PNR', 'TDG', 'URI'], dtype=object),
 'Health Care': array(['ALGN', 'CTLT', 'XRAY', 'GEHC', 'WST'], dtype=object),
 'Information Technology': array(['ANSS', 'MPWR', 'NVDA', 'STX', 'SWKS'], dtype=object),
 'Communication Services': array(['FOXA', 'META', 'OMC', 'PARA', 'WBD'], dtype=object),
 'Consumer Staples': array(['CHD', 'CLX', 'EL', 'HSY', 'LW'], dtype=object),
 'Consumer Discretionary': array(['BKNG', 'LVS', 'PHM', 'RCL', 'WYNN'], dtype=object),
 'Utilities': array(['EIX', 'EVRG', 'EXC', 'PCG', 'PEG'], dtype=object),
 'Financials': array(['ACGL', 'RE', 'MKTX', 'STT', 'SIVB'], dtype=object),
 'Materials': array(['CE', 'DOW', 'LYB', 'NUE', 'STLD'], dtype=object),
 'Real Estate': array(['CBRE', 'ESS', 'PLD', 'VTR', 'WELL'], dtype=object),
 'Energy': array(['BKR', 'MPC', 'SLB', 'TRGP', 'VLO'], dtype=object)}

In [41]:
# Line chart
line_chart = alt.Chart(df_top_5_communication).mark_line().encode(
    x='Date:T',
    y='Close:Q',
    color='Symbol:N',
    tooltip=['Symbol:N', 'Close:Q']
).properties(
    width=800,
    height=400,
    title='Top 5 Companies by Growth Rate'
).interactive()

line_chart

alt.Chart(...)

In [42]:
alt.Chart(df_top_5_communication).mark_arc().encode(
    color='Symbol:N',
    theta='Volume:Q'
)

alt.Chart(...)

In [27]:
# Define a function of line chart with multiple lines to show the trend of top 5 companies in each sector

def line_chart_growth_rate(duration):
    company_growth_rate = growth_rate_company(duration)

    # Filter the top 5 companies in each sector
    top_5 = company_growth_rate.groupby('sector').apply(lambda x: x.nlargest(5, 'growth_rate'))
    top_5 = top_5.reset_index(drop=True)

    # Obtain the symbol of top 5 companies in each sector
    top_5_symbol = []
    for i in sector_name:
        top_5_symbol.append(top_5[top_5['sector']==i]['company'].unique().tolist())

    # Subset the df to only include the top 5 companies in each sector
    df_top_5 = df[df['Symbol'].isin(top_5_symbol[0]+top_5_symbol[1]+top_5_symbol[2]+top_5_symbol[3]+top_5_symbol[4]+top_5_symbol[5]+top_5_symbol[6]+top_5_symbol[7]+top_5_symbol[8]+top_5_symbol[9]+top_5_symbol[10])]

    # Subset the df_top_5 based on the GICS Sector
    # for example, if we want to see the top 5 companies in Communication Services sector
    df_top_5_communication = df_top_5[df_top_5['GICS Sector']=='Communication Services']
    df_top_5_communication

    # Line chart
    line_chart = alt.Chart(df_top_5_communication).mark_line().encode(
        x='Date:T',
        y='Close:Q',
        color='Symbol:N',
        tooltip=['Symbol:N', 'Close:Q']
    ).properties(
        width=800,
        height=400,
        title='Top 5 Companies by Growth Rate'
    ).interactive()

    return line_chart


In [28]:
line_chart_growth_rate(90)

TypeError: growth_rate_company() missing 1 required positional argument: 'df'

In [ ]:
# def update_company_options(selected_sector):

df_top_5_selected_sector = df_top_5[df_top_5['GICS Sector'] == 'Communication Services']
top_5_symbol = df_top_5_selected_sector['Symbol'].unique().tolist()
options = [{'label': symbol, 'value': symbol} for symbol in top_5_symbol]

options

In [ ]:
# def update_trend_line(selected_sector, selected_companies):

df_top_5_selected_sector = df_top_5[df_top_5['GICS Sector'] == 'Communication Services']
df_top_5_selected_sector_company = df_top_5_selected_sector[df_top_5_selected_sector['Symbol'] == 'NFLX']

graph = alt.Chart(df_top_5_selected_sector_company).mark_line().encode(
    x='Date:T',
    y='Close:Q',
    color='Symbol:N',
    tooltip=['Symbol:N', 'Close:Q']
).properties(
    width=800,
    height=400,
    title='Top 5 Companies by Growth Rate'
).interactive()

graph



In [ ]:
import plotly
import plotly.express as px
px.line(df_top_5_selected_sector_company, x='Date', y='Close', color='Symbol')

In [ ]:
# create dash app
app = dash.Dash(__name__)

# Define the dropdown for sector selection
sectors = df_top_5['GICS Sector'].unique().tolist()
dropdown_sector = dcc.Dropdown(
    id='dropdown_sector',
    options=[{'label': sector, 'value': sector} for sector in sectors],
    value=sectors[0]
)

# Define the checkboxes for company selection
checkbox_company = dcc.Checklist(
    id='checkbox_company',
    options=[],
    value=[]
)

# Define the graph for the trend line plot
graph_trend_line = dcc.Graph(id='graph_trend_line')

# Define the layout of the app
app.layout = html.Div([
    dropdown_sector,
    checkbox_company,
    graph_trend_line
])

# Define the callback to update the options of the checkbox based on the selected sector
@app.callback(
    Output('checkbox_company', 'options'),
    Input('dropdown_sector', 'value')
)
def update_company_options(selected_sector):
    df_top_5_selected_sector = df_top_5[df_top_5['GICS Sector'] == selected_sector]
    top_5_symbol = df_top_5_selected_sector['Symbol'].unique().tolist()
    options = [{'label': symbol, 'value': symbol} for symbol in top_5_symbol]
    return options

# Define the callback to update the graph based on the selected sector and companies
@app.callback(
    Output('graph_trend_line', 'figure'),
    [Input('dropdown_sector', 'value'),
     Input('checkbox_company', 'value')]
)
def update_trend_line(selected_sector, selected_companies):
    df_top_5_selected_sector = df_top_5[df_top_5['GICS Sector'] == selected_sector]
    df_top_5_selected_sector_company = df_top_5_selected_sector[df_top_5_selected_sector['Symbol'] == selected_companies]

    graph = alt.Chart(df_top_5_selected_sector_company).mark_line().encode(
        x='Date:T',
        y='Close:Q',
        color='Symbol:N',
        tooltip=['Symbol:N', 'Close:Q']
    ).properties(
        width=800,
        height=400,
        title='Top 5 Companies by Growth Rate'
    )

    return graph


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


In [ ]:
# create dash app
app = dash.Dash(__name__)

# Define the dropdown for sector selection
sectors = df_top_5['GICS Sector'].unique().tolist()
dropdown_sector = dcc.Dropdown(
    id='dropdown_sector',
    options=[{'label': sector, 'value': sector} for sector in sectors],
    value=sectors[0]
)

# Define the checkboxes for company selection
checkbox_company = dcc.Checklist(
    id='checkbox_company',
    options=[],
    value=[]
)

# Define the graph for the trend line plot
# graph_trend_line = dcc.Graph(id='graph_trend_line')

# Define the layout of the app
app.layout = html.Div([
    html.Iframe(
        id='scatter',
        style={'border-width': '0', 'width': '100%', 'height': '400px'}),
    dropdown_sector,
    checkbox_company
])


# Define the callback to update the options of the checkbox based on the selected sector
@app.callback(
    Output('checkbox_company', 'options'),
    Input('dropdown_sector', 'value')
)
def update_company_options(selected_sector):
    df_top_5_selected_sector = df_top_5[df_top_5['GICS Sector'] == selected_sector]
    top_5_symbol = df_top_5_selected_sector['Symbol'].unique().tolist()
    options = [{'label': symbol, 'value': symbol} for symbol in top_5_symbol]
    return options


# Define the callback to update the graph based on the selected sector and companies
@app.callback(
    Output('scatter', 'srcDoc'),
    [Input('dropdown_sector', 'value'),
     Input('checkbox_company', 'value')]
)
def update_trend_line(selected_sector, selected_companies):
    df_top_5_selected_sector = df_top_5[df_top_5['GICS Sector'] == selected_sector]
    df_top_5_selected_sector_company = df_top_5_selected_sector[
        df_top_5_selected_sector['Symbol'] == selected_companies]

    graph = alt.Chart(df_top_5_selected_sector_company).mark_line().encode(
        x='Date:T',
        y='Close:Q',
        color='Symbol:N',
        tooltip=['Symbol:N', 'Close:Q']
    ).properties(
        width=800,
        height=400,
        title='Top 5 Companies by Growth Rate'
    ).interactive()
    return graph.to_html()


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
# create dash app
app = dash.Dash(__name__)

# Define the dropdown for sector selection
sectors = df_top_5['GICS Sector'].unique().tolist()
dropdown_sector = dcc.Dropdown(
    id='dropdown_sector',
    options=[{'label': sector, 'value': sector} for sector in sectors],
    value=sectors[0]
)

# Define the checkboxes for company selection
checkbox_company = dcc.Checklist(
    id='checkbox_company',
    options=[],
    value=[]
)

# Define the graph for the trend line plot
graph_trend_line = dcc.Graph(id='graph_trend_line')

# Define the layout of the app
app.layout = html.Div([
    dropdown_sector,
    checkbox_company,
    graph_trend_line
])


# Define the callback to update the options of the checkbox based on the selected sector
@app.callback(
    Output('checkbox_company', 'options'),
    Input('dropdown_sector', 'value')
)
def update_company_options(selected_sector):
    df_top_5_selected_sector = df_top_5[df_top_5['GICS Sector'] == selected_sector]
    top_5_symbol = df_top_5_selected_sector['Symbol'].unique().tolist()
    options = [{'label': symbol, 'value': symbol} for symbol in top_5_symbol]
    return options


# Define the callback to update the graph based on the selected sector and companies
@app.callback(
    Output('graph_trend_line', 'figure'),
    [Input('dropdown_sector', 'value'),
     Input('checkbox_company', 'value')]
)
def update_trend_line(selected_sector, selected_companies):
    df_top_5_selected_sector = df_top_5[df_top_5['GICS Sector'] == selected_sector]
    df_top_5_selected_sector_company = df_top_5_selected_sector[
        df_top_5_selected_sector['Symbol'] == selected_companies]

    graph = px.line(df_top_5_selected_sector_company, x='Date', y='Close', color='Symbol')
    return graph


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
## backup
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output, State
import altair as alt
import dash_bootstrap_components as dbc
import pandas as pd
import plotly
import plotly.express as px

# Read in global data
df = pd.read_csv('data/SP500_merged.csv')

sector_name = df['GICS Sector'].unique().tolist()

sector_symbol = []
for i in sector_name:
    sector_symbol.append(df[df['GICS Sector'] == i]['Symbol'].unique().tolist())


def growth_rate_company(duration):
    company_growth_rate = []

    for i in range(len(sector_name)):
        grow_rate = []

        for n in sector_symbol[i]:
            dff = df[df['GICS Sector'] == sector_name[i]][df['Symbol'] == n]
            grow_rate.append(dff.iloc[len(dff) - 1, 1] / dff.iloc[len(dff) - 1 - duration, 1] - 1)

        company_growth_rate.extend([{'sector': sector_name[i],
                                     'company': n, 'growth_rate': g
                                     } for n, g in zip(sector_symbol[i], grow_rate)])

    company_growth_rate = pd.DataFrame(company_growth_rate)

    return company_growth_rate


company_growth_rate = growth_rate_company(90)

top_5 = company_growth_rate.groupby('sector').apply(lambda x: x.nlargest(5, 'growth_rate'))
top_5 = top_5.reset_index(drop=True)

top_5_symbol = []
for i in sector_name:
    top_5_symbol.append(top_5[top_5['sector'] == i]['company'].unique().tolist())

df_top_5 = df[df['Symbol'].isin(top_5_symbol[0] + top_5_symbol[1] + top_5_symbol[2] + top_5_symbol[3] +
                                top_5_symbol[4] + top_5_symbol[5] + top_5_symbol[6] + top_5_symbol[7] +
                                top_5_symbol[8] + top_5_symbol[9] + top_5_symbol[10])]

# remove Healthcare sector
df_top_5 = df_top_5[df_top_5['GICS Sector'] != 'Health Care']


# Define the sectors and their symbols
sector_name = df_top_5['GICS Sector'].unique().tolist()
symbols_by_sector = {sector: df_top_5[df_top_5['GICS Sector'] == sector]['Symbol'].unique() for sector in sector_name}

#####


# create dash app
app = dash.Dash(__name__)

# Define the dropdown for sector selection
sectors = df_top_5['GICS Sector'].unique().tolist()
dropdown_sector = dcc.Dropdown(
    id='dropdown_sector',
    options=[{'label': sector, 'value': sector} for sector in sectors],
    value=sectors[0]
)

# Define the checkboxes for company selection
checkbox_company = dcc.Checklist(
    id='checkbox_company',
    options=[],
    value=[]
)

# Define the graph for the trend line plot
# graph_trend_line = dcc.Graph(id='graph_trend_line')

# Define the layout of the app
app.layout = html.Div([
    html.Iframe(
        id='scatter',
        style={'border-width': '0', 'width': '100%', 'height': '400px'}),
    dropdown_sector,
    checkbox_company
])


# Define the callback to update the options of the checkbox based on the selected sector
@app.callback(
    Output('checkbox_company', 'options'),
    Input('dropdown_sector', 'value')
)
def update_company_options(selected_sector):
    df_top_5_selected_sector = df_top_5[df_top_5['GICS Sector'] == selected_sector]
    top_5_symbol = df_top_5_selected_sector['Symbol'].unique().tolist()
    options = [{'label': symbol, 'value': symbol} for symbol in top_5_symbol]
    return options


# Define the callback to update the graph based on the selected sector and companies
@app.callback(
    Output('scatter', 'srcDoc'),
    [Input('dropdown_sector', 'value'),
     Input('checkbox_company', 'value')]
)
def update_trend_line(selected_sector, selected_companies):
    df_top_5_selected_sector = df_top_5[df_top_5['GICS Sector'] == selected_sector]

    if not selected_companies:
        # If no companies are selected, return an empty dataframe
        df_top_5_selected_sector_company = pd.DataFrame(columns=df_top_5.columns)
    else:
        # Filter the dataframe based on the selected companies
        df_top_5_selected_sector_company = df_top_5_selected_sector[
            df_top_5_selected_sector['Symbol'].isin(selected_companies)]

    chart = alt.Chart(df_top_5_selected_sector_company).mark_line().encode(
        x='Date:T',
        y='Close:Q',
        color='Symbol:N',
        tooltip=['Symbol:N', 'Close:Q']
    ).properties(
        width=800,
        height=400,
        title='Top 5 Companies by Growth Rate'
    ).interactive()
    return chart.to_html()


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
